In [ ]:
from  langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
import os
from langraph.graph import Graph

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langgraph.graph import Graph

import sqlite3
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langgraph.graph import Graph

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", openai_api_key="your_api_key")

# Memory for storing claim history
memory = ConversationBufferMemory(memory_key="claim_history")

# Database Setup
def initialize_database():
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS claims (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            claim_text TEXT,
            category TEXT,
            fraud_status TEXT,
            risk_level TEXT,
            payout_estimate TEXT,
            policy_validation TEXT,
            final_decision TEXT
        )
    """)
    conn.commit()
    conn.close()

# Store claim data in database
def save_claim_to_db(claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision):
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO claims (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision))
    conn.commit()
    conn.close()

    # Retrieve claim from database if it was processed before
def fetch_claim_from_db(claim_text):
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM claims WHERE claim_text = ?", (claim_text,))
    result = cursor.fetchone()
    conn.close()
    return result

# Initialize Database
initialize_database()

# 1️⃣ Categorization Function
def claim_categorization(claim):
    """Categorizes the insurance claim into types (e.g., auto, health, property)."""
    response = llm.predict(f"Classify this insurance claim: {claim}")
    memory.save_context({"claim": claim}, {"category": response})
    return response

# 2️⃣ Fraud Detection Function
def fraud_detection(claim_category):
    """Analyzes if the claim is potentially fraudulent."""
    response = llm.predict(f"Determine if this {claim_category} claim is fraudulent.")
    memory.save_context({"claim_category": claim_category}, {"fraud_status": response})
    return response

# 3️⃣ Risk Assessment Function
def risk_assessment(fraud_status):
    """Assesses risk level based on claim category and fraud status."""
    response = llm.predict(f"Assess risk level for a claim flagged as {fraud_status}.")
    memory.save_context({"fraud_status": fraud_status}, {"risk_level": response})
    return response

# 4️⃣ Payout Estimation Function
def payout_estimation(risk_level):
    """Estimates claim payout based on risk level."""
    response = llm.predict(f"Estimate the payout amount for a {risk_level} risk claim.")
    memory.save_context({"risk_level": risk_level}, {"payout_estimate": response})
    return response

# 5️⃣ Policy Validation Function
def policy_validation(claim_details):
    """Checks whether the claim adheres to policy guidelines."""
    response = llm.predict(f"Validate if this claim meets policy requirements: {claim_details}")
    memory.save_context({"claim_details": claim_details}, {"policy_validation": response})
    return response

# 6️⃣ Final Approval Function
def final_approval(payout_estimate):
    """Decides if the claim should be approved or denied based on all prior analysis."""
    response = llm.predict(f"Based on a payout estimate of {payout_estimate}, should this claim be approved?")
    memory.save_context({"payout_estimate": payout_estimate}, {"final_decision": response})
    return response

# Define Tools for Each Agent
categorization_tool = Tool(name="Claim Categorization", func=claim_categorization, description="Classifies an insurance claim.")
fraud_detection_tool = Tool(name="Fraud Detection", func=fraud_detection, description="Checks for fraud in a classified claim.")
risk_assessment_tool = Tool(name="Risk Assessment", func=risk_assessment, description="Evaluates risk based on claim details.")
payout_estimation_tool = Tool(name="Payout Estimation", func=payout_estimation, description="Estimates claim payout based on severity.")
policy_validation_tool = Tool(name="Policy Validation", func=policy_validation, description="Validates if the claim aligns with policy terms.")
final_approval_tool = Tool(name="Final Approval", func=final_approval, description="Determines whether to approve or deny a claim.")

# Define Agents with Memory & Different Types
categorization_agent = initialize_agent(
    tools=[categorization_tool], llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True
)
fraud_detection_agent = initialize_agent(
    tools=[fraud_detection_tool], llm=llm, agent=AgentType.REACT_DOCSTORE, memory=memory, verbose=True
)
risk_assessment_agent = initialize_agent(
    tools=[risk_assessment_tool], llm=llm, agent=AgentType.SELF_ASK_WITH_SEARCH, memory=memory, verbose=True
)
payout_estimation_agent = initialize_agent(
    tools=[payout_estimation_tool], llm=llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True
)
policy_validation_agent = initialize_agent(
    tools=[policy_validation_tool], llm=llm, agent=AgentType.OPENAI_FUNCTIONS, memory=memory, verbose=True
)
final_approval_agent = initialize_agent(
    tools=[final_approval_tool], llm=llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True
)

# Create Multi-RAG AI Workflow
graph = Graph()

# Add Nodes for Each Agent
graph.add_node("categorization_agent", categorization_agent.run)
graph.add_node("fraud_detection_agent", fraud_detection_agent.run)
graph.add_node("risk_assessment_agent", risk_assessment_agent.run)
graph.add_node("payout_estimation_agent", payout_estimation_agent.run)
graph.add_node("policy_validation_agent", policy_validation_agent.run)
graph.add_node("final_approval_agent", final_approval_agent.run)

# Define Execution Flow
graph.add_edge("categorization_agent", "fraud_detection_agent")
graph.add_edge("fraud_detection_agent", "risk_assessment_agent")
graph.add_edge("risk_assessment_agent", "payout_estimation_agent")
graph.add_edge("payout_estimation_agent", "policy_validation_agent")
graph.add_edge("policy_validation_agent", "final_approval_agent")

# Compile the Workflow
workflow = graph.compile()

# Store the results in the database
    save_claim_to_db(
        inputs["claim_text"], inputs["category"], inputs["fraud_status"],
        inputs["risk_level"], inputs["payout_estimate"], inputs["policy_validation"], response
    )

    return {"final_decision": response}

# Function to Handle User Queries
def process_user_query(user_input):
    """
    Determines whether the user input is a new claim, a status request, or a generic question.
    """
    # Check if user is asking about an existing claim
    previous_claim = fetch_claim_from_db(user_input)
    if previous_claim:
        print("\n🔹 Retrieved Claim from Database:")
        print(f"Category: {previous_claim[2]}, Fraud Status: {previous_claim[3]}, Risk Level: {previous_claim[4]}")
        print(f"Payout Estimate: {previous_claim[5]}, Policy Validation: {previous_claim[6]}, Final Decision: {previous_claim[7]}")
        return

    # If no prior claim exists, process a new claim
    print("\n🔹 Processing New Claim:", user_input)
    result = workflow.invoke({"claim_text": user_input})

    # Display Final Decision
    print("\n🔹 Processed Claim Result:")
    print(f"Category: {result['category']}")
    print(f"Fraud Status: {result['fraud_status']}")
    print(f"Risk Level: {result['risk_level']}")
    print(f"Payout Estimate: {result['payout_estimate']}")
    print(f"Policy Validation: {result['policy_validation']}")
    print(f"Final Decision: {result['final_decision']}")

# 🔹 Simulating User Input
while True:
    user_input = input("\n🔹 Enter a claim description or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        break
    process_user_query(user_input)

In [ ]:
import sqlite3
import openai
import random

# Set your OpenAI API key
openai.api_key = "your_openai_api_key"

# Function to query OpenAI's GPT-4 for decision-making
def query_llm(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an expert insurance claims processor."},
                  {"role": "user", "content": prompt}],
        max_tokens=100
    )
    return response['choices'][0]['message']['content'].strip()

# 🔹 Define AI Agents
class ReactiveAgent:
    """Agent that immediately processes claims without memory or learning."""
    def process_claim(self, claim_text):
        category = query_llm(f"Categorize this insurance claim: {claim_text}")
        return f"Categorized as {category}"

class ReflectionAgent:
    """Agent that improves claim categorization over time."""
    def __init__(self):
        self.memory = {}

    def process_claim(self, claim_text):
        if claim_text in self.memory:
            return f"Using learned category: {self.memory[claim_text]}"
        category = query_llm(f"Categorize this insurance claim: {claim_text}")
        self.memory[claim_text] = category
        return f"Categorized as {category}"

class ReActAgent:
    """Agent that reasons before acting."""
    def reason(self, claim_text):
        return query_llm(f"Analyze this claim before making a decision: {claim_text}")

    def act(self, claim_text):
        return query_llm(f"Now process this claim based on your reasoning: {claim_text}")

    def process_claim(self, claim_text):
        reasoning = self.reason(claim_text)
        action = self.act(claim_text)
        return f"{reasoning}\n{action}"

class ReWOOAgent:
    """Agent that decides whether to observe data or use prior knowledge."""
    def __init__(self):
        self.internal_knowledge = {}

    def observe(self, claim_text):
        return query_llm(f"Retrieve external fraud data for this claim: {claim_text}")

    def reason(self, claim_text):
        return query_llm(f"Reason internally about this claim without external data: {claim_text}")

    def process_claim(self, claim_text):
        if random.choice([True, False]):  # Randomly decides to observe or reason
            return self.observe(claim_text)
        else:
            return self.reason(claim_text)

class DeliberativeAgent:
    """Agent that plans before making a decision."""
    def plan(self, claim_text):
        return query_llm(f"Plan a step-by-step process to assess this claim: {claim_text}")

    def process_claim(self, claim_text):
        plan = self.plan(claim_text)
        final_decision = query_llm(f"Based on your plan, what is the final decision for this claim: {claim_text}")
        return f"Plan: {plan}\nFinal Decision: {final_decision}"

class LearningAgent:
    """Agent that learns fraud patterns over time."""
    def __init__(self):
        self.fraud_memory = {}

    def detect_fraud(self, claim_text):
        if claim_text in self.fraud_memory:
            return f"Using learned fraud pattern: {self.fraud_memory[claim_text]}"
        fraud_status = query_llm(f"Analyze if this claim is fraudulent: {claim_text}")
        self.fraud_memory[claim_text] = fraud_status
        return f"Fraud Analysis: {fraud_status}"

class HybridAgent:
    """Combines memory, reasoning, learning, and planning for claim processing."""
    def __init__(self):
        self.memory = {}

    def learn(self, claim_text, category):
        self.memory[claim_text] = category

    def reflect(self, claim_text):
        return self.memory.get(claim_text, "Unknown, needs processing.")

    def plan(self, claim_text):
        return query_llm(f"Plan a structured process for assessing this claim: {claim_text}")

    def process_claim(self, claim_text):
        if claim_text in self.memory:
            return self.reflect(claim_text)
        category = query_llm(f"Categorize this claim: {claim_text}")
        self.learn(claim_text, category)
        return f"Planned Process -> Learned: {category}"

# 🔹 Database Setup
def initialize_database():
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS claims (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            claim_text TEXT,
            category TEXT,
            fraud_status TEXT,
            risk_level TEXT,
            payout_estimate TEXT,
            policy_validation TEXT,
            final_decision TEXT
        )
    """)
    conn.commit()
    conn.close()

# 🔹 Store claim in database
def save_claim_to_db(claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision):
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO claims (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision))
    conn.commit()
    conn.close()

# 🔹 Retrieve claim from database
def fetch_claim_from_db(claim_text):
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM claims WHERE claim_text = ?", (claim_text,))
    result = cursor.fetchone()
    conn.close()
    return result

# 🔹 Initialize Database
initialize_database()

# 🔹 Assign Agents
agents = {
    "Reactive": ReactiveAgent(),
    "Reflection": ReflectionAgent(),
    "ReAct": ReActAgent(),
    "ReWOO": ReWOOAgent(),
    "Deliberative": DeliberativeAgent(),
    "Learning": LearningAgent(),
    "Hybrid": HybridAgent()
}

# 🔹 Process Claim with Selected Agent
def process_user_query(user_input, agent_type):
    if agent_type not in agents:
        print("Invalid agent type.")
        return

    agent = agents[agent_type]

    # Check if claim exists
    previous_claim = fetch_claim_from_db(user_input)
    if previous_claim:
        print("\n🔹 Retrieved Claim from Database:")
        print(f"Category: {previous_claim[2]}, Fraud Status: {previous_claim[3]}, Risk Level: {previous_claim[4]}")
        print(f"Payout Estimate: {previous_claim[5]}, Policy Validation: {previous_claim[6]}, Final Decision: {previous_claim[7]}")
        return

    # Process claim using LLM-powered agent
    print(f"\n🔹 Processing with {agent_type} Agent:")
    result = agent.process_claim(user_input)
    print(result)

    # Simulated Outputs
    category = query_llm(f"Categorize this claim: {user_input}")
    fraud_status = query_llm(f"Is this claim fraudulent? {user_input}")
    risk_level = query_llm(f"Assess the risk level of this claim: {user_input}")
    payout_estimate = query_llm(f"Estimate payout for this claim: {user_input}")
    policy_validation = query_llm(f"Validate if this claim adheres to policy: {user_input}")
    final_decision = query_llm(f"Should this claim be approved? {user_input}")

    # Save to DB
    save_claim_to_db(user_input, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision)

# 🔹 User Input Simulation
while True:
    user_input = input("\n🔹 Enter a claim description or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        break
    agent_type = input("Select agent type (Reactive, Reflection, ReAct, ReWOO, Deliberative, Learning, Hybrid): ")
    process_user_query(user_input, agent_type)
